In [146]:
import findspark
findspark.init('/opt/spark')

In [147]:
from pyspark.sql import SparkSession

In [148]:
spark = SparkSession.builder\
    .getOrCreate()

spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

In [4]:
! hdfs dfs -mkdir /tmp/boston_crimes

In [5]:
! hdfs dfs -put crime.csv /tmp/boston_crimes

In [8]:
! hdfs dfs -put offense_codes.csv /tmp/boston_crimes

In [149]:
! hdfs dfs -ls /tmp/boston_crimes

Found 2 items
-rw-r--r--   2 jupyter supergroup   57969510 2024-12-26 22:48 /tmp/boston_crimes/crime.csv
-rw-r--r--   2 jupyter supergroup      19406 2024-12-26 23:15 /tmp/boston_crimes/offense_codes.csv


In [150]:
CrimeBoston = spark.read.option("header", "true").option("inferSchema", "true").csv("/tmp/boston_crimes/crime.csv")
CrimeBoston.show()

+---------------+------------+--------------------+--------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+-----------------+-----------+------------+--------------------+
|INCIDENT_NUMBER|OFFENSE_CODE|  OFFENSE_CODE_GROUP| OFFENSE_DESCRIPTION|DISTRICT|REPORTING_AREA|SHOOTING|   OCCURRED_ON_DATE|YEAR|MONTH|DAY_OF_WEEK|HOUR|  UCR_PART|           STREET|        Lat|        Long|            Location|
+---------------+------------+--------------------+--------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+-----------------+-----------+------------+--------------------+
|     I182070945|         619|             Larceny|  LARCENY ALL OTHERS|     D14|           808|    null|2018-09-02 13:00:00|2018|    9|     Sunday|  13|  Part One|       LINCOLN ST|42.35779134|-71.13937053|(42.35779134, -71...|
|     I182070943|        1402|           Vandalism|           VANDALISM|     C11|   

In [151]:
OfCodes = spark.read.option("header", "true").option("inferSchema", "true").csv("/tmp/boston_crimes/offense_codes.csv")
OfCodes.show()

+----+--------------------+
|CODE|                NAME|
+----+--------------------+
| 612|LARCENY PURSE SNA...|
| 613| LARCENY SHOPLIFTING|
| 615|LARCENY THEFT OF ...|
|1731|              INCEST|
|3111|LICENSE PREMISE V...|
|2646|LIQUOR - DRINKING...|
|2204|LIQUOR LAW VIOLATION|
|3810|M/V ACCIDENT - IN...|
|3801|M/V ACCIDENT - OTHER|
|3807|M/V ACCIDENT - OT...|
|3803|M/V ACCIDENT - PE...|
|3805|M/V ACCIDENT - PO...|
|3802|M/V ACCIDENT - PR...|
|3205|   M/V PLATES - LOST|
| 123|MANSLAUGHTER - NO...|
| 121|MANSLAUGHTER - VE...|
|3501|      MISSING PERSON|
|3502|MISSING PERSON - ...|
|3503|MISSING PERSON - ...|
| 111|MURDER, NON-NEGLI...|
+----+--------------------+
only showing top 20 rows



In [93]:
#общее количество преступлений в районе

In [285]:
from pyspark.sql.functions import concat, lit

In [286]:
CrimeStats = CrimeBoston.groupBy('DISTRICT','YEAR','MONTH').count().withColumnRenamed('count', 'crimes_total')

In [287]:
CrimeStats.createOrReplaceTempView("CrimeStats_df")

In [289]:
CrimeStatsTotal = spark.sql("SELECT DISTRICT, sum(crimes_total) as crimes_total from CrimeStats_df group by DISTRICT order by DISTRICT desc")

In [290]:
CrimeStatsTotal.show()

+--------+------------+
|DISTRICT|crimes_total|
+--------+------------+
|      E5|       13239|
|     E18|       17348|
|     E13|       17536|
|      D4|       41915|
|     D14|       20127|
|      C6|       23460|
|     C11|       42530|
|      B3|       35442|
|      B2|       49945|
|      A7|       13544|
|     A15|        6505|
|      A1|       35717|
|    null|        1765|
+--------+------------+



In [226]:
CrimeStatsMonthly = CrimeStats.withColumn('month_year', (concat(col('MONTH'), lit("_"), col('YEAR'))))

In [227]:
CrimeStatsMonthly.show()

+--------+----+-----+------------+----------+
|DISTRICT|YEAR|MONTH|crimes_total|month_year|
+--------+----+-----+------------+----------+
|     C11|2015|    9|        1147|    9_2015|
|     A15|2016|    6|         203|    6_2016|
|      C6|2015|   10|         593|   10_2015|
|     E13|2017|    4|         439|    4_2017|
|      B3|2015|   11|         845|   11_2015|
|     C11|2017|    2|        1048|    2_2017|
|      E5|2015|   12|         335|   12_2015|
|      A7|2016|    2|         309|    2_2016|
|      A7|2015|   11|         356|   11_2015|
|      B2|2015|   10|        1373|   10_2015|
|      B3|2015|   10|         835|   10_2015|
|      D4|2017|    4|        1077|    4_2017|
|      C6|2016|    5|         585|    5_2016|
|     A15|2016|    3|         155|    3_2016|
|     E13|2015|    9|         468|    9_2015|
|     A15|2015|    8|         150|    8_2015|
|     D14|2016|   10|         559|   10_2016|
|      D4|2015|   10|        1072|   10_2015|
|     E18|2016|    1|         410|

In [228]:
#медиана числа преступлений в месяц в районе

In [274]:
CrimeStatsMonthly.createOrReplaceTempView("CrimeStatsMonthly2")

In [276]:
MedianMonthlyStats = spark.sql("SELECT DISTRICT, median(crimes_total) as crimes_monthly from CrimeStatsMonthly2 group by DISTRICT order by DISTRICT desc")

In [277]:
MedianMonthlyStats.show()

+--------+--------------+
|DISTRICT|crimes_monthly|
+--------+--------------+
|      E5|         337.5|
|     E18|         437.5|
|     E13|         446.5|
|      D4|        1086.0|
|     D14|         510.5|
|      C6|         595.5|
|     C11|        1117.0|
|      B3|         913.0|
|      B2|        1301.0|
|      A7|         345.0|
|     A15|         162.5|
|      A1|         909.5|
|    null|          41.0|
+--------+--------------+



In [233]:
#координаты районов

In [250]:
CrimePlace = CrimeBoston.groupBy('DISTRICT').avg('lat','Long').withColumnRenamed('avg(lat)', 'lat').withColumnRenamed('avg(Long)', 'lng')

In [251]:
CrimePlace.show()

+--------+------------------+------------------+
|DISTRICT|               lat|               lng|
+--------+------------------+------------------+
|      C6| 42.21212258445509|-70.85561011772268|
|    null| 25.23950519369344|-43.44877438704257|
|      B2| 42.31600367732632| -71.0756993065438|
|     C11| 42.29263740899965|-71.05125995734306|
|     E13|42.309803655709324|-71.09800478878299|
|      B3| 42.28305944520085|-71.07894914185519|
|      E5|42.197969994470235|-71.00440862434776|
|     A15|42.179155250910874|-70.74472508958492|
|      A7| 42.36070260499406| -71.0039483303988|
|     D14|  42.3435072451093|-71.13125461726422|
|      D4|42.341242517909265|-71.07725024947149|
|     E18|  42.2626806112253| -71.1189199875768|
|      A1| 42.33123077259858|-71.01991881361955|
+--------+------------------+------------------+



In [ ]:
#объединение

In [294]:
BostonCrimeTotal1 = (CrimePlace.alias('a')
                .join(MedianMonthlyStats.alias('b'), MedianMonthlyStats["DISTRICT"] == CrimePlace["DISTRICT"])).select('a.DISTRICT', 'b.crimes_monthly', 'a.lat', 'a.lng')

BostonCrimeTotal1.show()

+--------+--------------+------------------+------------------+
|DISTRICT|crimes_monthly|               lat|               lng|
+--------+--------------+------------------+------------------+
|      A1|         909.5| 42.33123077259858|-71.01991881361955|
|     A15|         162.5|42.179155250910874|-70.74472508958492|
|      A7|         345.0| 42.36070260499406| -71.0039483303988|
|      B2|        1301.0| 42.31600367732632| -71.0756993065438|
|      B3|         913.0| 42.28305944520085|-71.07894914185519|
|     C11|        1117.0| 42.29263740899965|-71.05125995734306|
|      C6|         595.5| 42.21212258445509|-70.85561011772268|
|     D14|         510.5|  42.3435072451093|-71.13125461726422|
|      D4|        1086.0|42.341242517909265|-71.07725024947149|
|     E13|         446.5|42.309803655709324|-71.09800478878299|
|     E18|         437.5|  42.2626806112253| -71.1189199875768|
|      E5|         337.5|42.197969994470235|-71.00440862434776|
+--------+--------------+---------------

In [296]:
BostonCrimeTotal = (BostonCrimeTotal1.alias('a').join(CrimeStatsTotal.alias('b'), CrimeStatsTotal["DISTRICT"] == BostonCrimeTotal1["DISTRICT"])).select('a.DISTRICT', 'b.crimes_total', 'a.crimes_monthly', 'a.lat', 'a.lng').orderBy('a.DISTRICT')

BostonCrimeTotal.show()

+--------+------------+--------------+------------------+------------------+
|DISTRICT|crimes_total|crimes_monthly|               lat|               lng|
+--------+------------+--------------+------------------+------------------+
|      A1|       35717|         909.5| 42.33123077259858|-71.01991881361955|
|     A15|        6505|         162.5|42.179155250910874|-70.74472508958492|
|      A7|       13544|         345.0| 42.36070260499406| -71.0039483303988|
|      B2|       49945|        1301.0| 42.31600367732632| -71.0756993065438|
|      B3|       35442|         913.0| 42.28305944520085|-71.07894914185519|
|     C11|       42530|        1117.0| 42.29263740899965|-71.05125995734306|
|      C6|       23460|         595.5| 42.21212258445509|-70.85561011772268|
|     D14|       20127|         510.5|  42.3435072451093|-71.13125461726422|
|      D4|       41915|        1086.0|42.341242517909265|-71.07725024947149|
|     E13|       17536|         446.5|42.309803655709324|-71.09800478878299|